In [42]:
import torch
from torch import tensor, nn
from loading_datas import  generate_pair_sets
import torch.nn.functional as F


In [43]:
torch.set_default_tensor_type(torch.cuda.FloatTensor if torch.cuda.is_available()
                              else torch.FloatTensor)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


train_pairs, train_target, train_classes, test_pairs, test_target, test_classes = generate_pair_sets(1000)
train_pairs = train_pairs.to(device)
train_target = train_target.to(device)
train_classes = train_classes.to(device)
test_pairs = test_pairs.to(device)
test_target = test_target.to(device)
test_classes = test_classes.to(device)

cuda


In [44]:
def compute_nb_errors(model, data_input, data_target, batch_size):

    nb_data_errors = 0

    for inputs, targets in zip(data_input.split(batch_size), data_target.split(batch_size)):
        output = model(inputs)
        # output = model(inputs)[0]
        for k in range(len(targets)):

            if torch.argmax(output[k]) != torch.argmax(targets[k]):
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors


In [45]:
class Net1(nn.Module):
  def __init__(self):
    super(Net1,self).__init__()
    #channel 1
    self.conv11 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv12 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv13 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv14 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv15 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    #channel 2
    self.conv21 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
    self.conv22 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(5, 5), padding=2)
    self.conv23 = nn.Conv2d(in_channels=32, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv24 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3, 3), padding=1)
    self.conv25 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1)

    self.maxPool = nn.MaxPool2d(kernel_size=(2, 2))



    self.fc1 = nn.Linear(512*3*3, 512)
    self.fc2 = nn.Linear(512, 64)
    self.fc3 = nn.Linear(64, 2)

  def forward(self,x): 
    # spliting the channels
    c1 = torch.narrow(x,1,0,1)
    c2 = torch.narrow(x,1,1,1)

    # Channel 1
    c1 = F.relu(self.conv11(c1))  #16 x 14 x 14
    c1 = F.relu(self.conv12(c1))  # 32 x 13 x 13
    c1 = self.maxPool(c1)
    c1 = F.relu(self.conv13(c1))  # 64 x 7 x 7
    c1 = F.relu(self.conv14(c1))  # 128 x 6 x 6
    c1 = F.relu(self.conv15(c1))  # 256 x 6 x 6
    c1 = self.maxPool(c1)  # 256 x 3 x 3

    # Channel 2
    c2 = F.relu(self.conv21(c2))  # 16 x 14 x 14
    c2 = F.relu(self.conv22(c2))  # 32 x 13 x 13
    c2 = self.maxPool(c2)
    c2 = F.relu(self.conv23(c2))  # 64 x 7 x 7
    c2 = F.relu(self.conv24(c2))  # 128 x 6 x 6
    c2 = F.relu(self.conv25(c2))  # 256 x 6 x 6
    c2 = self.maxPool(c2)  # 256 x 3 x 3





    output = torch.cat((c1,c2),1)

    output = output.view(-1,512*3*3)

    output = F.relu(self.fc1(output))
    output = F.relu(self.fc2(output))
    output = self.fc3(output)

    return output



In [46]:
lr, nb_epochs, batch_size = 1e-3, 100, 50
# model = Net1()
model = Net1()
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

model.to(device)
criterion.to(device)

for e in range(nb_epochs):
    for input, targets in zip(train_pairs.split(batch_size), train_target.split(batch_size)):
        # output, output_aux1, output_aux2 = model(input)
        output = model(input.to(device)).to(device)
        loss = criterion(output, targets.to(device))
        # loss_aux1 = criterion(output_aux1, targets)
        # loss_aux2 = criterion(output_aux2, targets)

        # print(loss)
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    train_errors = 100 * (1 - compute_nb_errors(model, train_pairs.to(device), train_target,batch_size)/train_pairs.size(0))
    test_errors = 100 * (1 - compute_nb_errors(model, test_pairs.to(device), test_target,batch_size)/test_pairs.size(0))
    print(f"Epoch # {e+1} / Train accuracy (%): {train_errors:.2f} / Test accuracy (%): {test_errors:.2f}")

        

Epoch # 1 / Train accuracy (%): 51.60 / Test accuracy (%): 48.20
Epoch # 2 / Train accuracy (%): 58.70 / Test accuracy (%): 55.90
Epoch # 3 / Train accuracy (%): 71.80 / Test accuracy (%): 67.30
Epoch # 4 / Train accuracy (%): 77.90 / Test accuracy (%): 75.60
Epoch # 5 / Train accuracy (%): 81.50 / Test accuracy (%): 78.50
Epoch # 6 / Train accuracy (%): 81.60 / Test accuracy (%): 80.00
Epoch # 7 / Train accuracy (%): 82.00 / Test accuracy (%): 80.20
Epoch # 8 / Train accuracy (%): 82.90 / Test accuracy (%): 80.40
Epoch # 9 / Train accuracy (%): 84.30 / Test accuracy (%): 81.80
Epoch # 10 / Train accuracy (%): 85.20 / Test accuracy (%): 82.20
Epoch # 11 / Train accuracy (%): 85.80 / Test accuracy (%): 82.50
Epoch # 12 / Train accuracy (%): 86.60 / Test accuracy (%): 83.10
Epoch # 13 / Train accuracy (%): 87.80 / Test accuracy (%): 83.20
Epoch # 14 / Train accuracy (%): 87.90 / Test accuracy (%): 83.20
Epoch # 15 / Train accuracy (%): 88.00 / Test accuracy (%): 83.20
Epoch # 16 / Train 

In [ ]:
output = model(test_pairs)
